# Capstone Two: Pre-Processing and Training Data<a id='4_Pre-Processing_and_Training_Data'></a>

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction import FeatureHasher
from sklearn.metrics import classification_report
from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

In [2]:
pokemon = pd.read_csv('Pokemon.csv')
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [3]:
pokemon.columns

Index(['#', 'Name', 'Type 1', 'Type 2', 'Total', 'HP', 'Attack', 'Defense',
       'Sp. Atk', 'Sp. Def', 'Speed', 'Generation', 'Legendary'],
      dtype='object')

In [4]:
pokemon['Type 1'].value_counts()

Water       112
Normal       98
Grass        70
Bug          69
Psychic      57
Fire         52
Electric     44
Rock         44
Dragon       32
Ground       32
Ghost        32
Dark         31
Poison       28
Steel        27
Fighting     27
Ice          24
Fairy        17
Flying        4
Name: Type 1, dtype: int64

In [5]:
pokemon['Type 2'].value_counts()

Flying      97
Ground      35
Poison      34
Psychic     33
Fighting    26
Grass       25
Fairy       23
Steel       22
Dark        20
Dragon      18
Water       14
Ghost       14
Ice         14
Rock        14
Fire        12
Electric     6
Normal       4
Bug          3
Name: Type 2, dtype: int64

In [6]:
pokemon["Type 2"] = pokemon["Type 2"].fillna(pokemon["Type 1"])

In [7]:
from sklearn import preprocessing

In [8]:
pokemon['single_type'] = (pokemon['Type 1']==pokemon['Type 2']).astype(int)

In [9]:
pokemon['single_type'].value_counts()

0    414
1    386
Name: single_type, dtype: int64

In [15]:
y=pokemon['single_type']
no_response_var_data=pokemon.drop(['Name', 'Type 1', 'Type 2', 'Generation', 'Legendary'],axis=1)
scaled=preprocessing.StandardScaler().fit(no_response_var_data)
scaled
X_scaled=scaled.transform(no_response_var_data)

In [16]:
no_response_var_data

,#,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,single_type
0,1,318,45,49,49,65,65,45,0
1,2,405,60,62,63,80,80,60,0
2,3,525,80,82,83,100,100,80,0
3,3,625,80,100,123,122,120,80,0
4,4,309,39,52,43,60,50,65,1
...,...,...,...,...,...,...,...,...,...
795,719,600,50,100,150,100,150,50,0
796,719,700,50,160,110,160,110,110,0
797,720,600,80,110,60,150,130,70,0
798,720,680,80,160,60,170,130,80,0


In [17]:
no_response_var_data.columns

Index(['#', 'Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed',
       'single_type'],
      dtype='object')

In [18]:
y=y.ravel()
X_train, X_test, y_train, y_tes=train_test_split(X_scaled,y)
print(no_response_var_data.columns)

Index(['#', 'Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed',
       'single_type'],
      dtype='object')


In [19]:
dumb= pokemon['Generation']
dummy_legend_values= pd.get_dummies(dumb)
dummy_legend_values
pokemon=pd.concat([pokemon,dummy_legend_values],axis=1)
del(pokemon['Generation'])
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Legendary,single_type,1,2,3,4,5,6
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,False,0,1,0,0,0,0,0
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,False,0,1,0,0,0,0,0
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,False,0,1,0,0,0,0,0
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,False,0,1,0,0,0,0,0
4,4,Charmander,Fire,Fire,309,39,52,43,60,50,65,False,1,1,0,0,0,0,0


In [ ]:
# Modeling

In [20]:
from sklearn import linear_model
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [21]:
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

In [22]:
y_prediction=model.predict(X_test)

In [23]:
explained_variance_score(y_prediction, y_tes)

1.0

In [24]:
mean_absolute_error(y_prediction, y_tes)

4.718447854656915e-16

In [25]:
pokemon.to_csv("pokemon_nv.csv")